<a href="https://colab.research.google.com/github/ssingh-rookie/tensorflow_nlp/blob/master/Tweet_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0

In [0]:
import tensorflow as tf

In [3]:
from google.colab import drive
root_dir = drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [0]:
!iconv -f LATIN1 -t UTF8 '/content/gdrive/My Drive/deeplearning/training.1600000.processed.noemoticon.csv' -o '/tmp/training_cleaned.csv'

In [0]:
import csv
corpus = []

i = 0 
with open('/tmp/training_cleaned.csv') as csvfile:
  reader = csv.reader(csvfile)
  #next(reader)

  for row in reader:
    list_item = []
    list_item.append(row[-1])
    label = row[0]
    if label == "0":
      list_item.append(0)
    else:
      list_item.append(1)
    
    corpus.append(list_item)
    i += 1
      



In [6]:
corpus[100:105]

[[' Body Of Missing Northern Calif. Girl Found: Police have found the remains of a missing Northern California girl .. http://tr.im/imji',
  0],
 ['@mangaaa I hope they will increase the capacity fast, yesterday was such a pain. Got the fail whale +15 times in 2 hours.... ',
  0],
 ['Behind on my classes for work ', 0],
 ['watching &quot;House&quot; ', 0],
 ['@kpreyes Remember my bum leg? Strikes back this time its serious ', 0]]

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [0]:
train_size = 160000
oov_token = "<OOV>"
trunc_type = "post"
pad_type = "post"
max_len = 32
test_split = 0.9
embedding_dim = 100

In [0]:
sub_sample = random.sample(corpus, k=train_size)

In [10]:
sub_sample[:5]

[["@evanl1 Don't tweet about food. I'm so hungry ", 0],
 ["@moeneekah ok, I have to like Mish it and try and finish my assingment tonight so I can't go out tomorrow  (that means no schoolll) yew ha",
  1],
 ['Just got a firmware update on his G1...looks good so far ', 1],
 ['omg! my stomach hurts so badly! i hope Lauren didnt give me what she had! ',
  0],
 ['http://twitpic.com/68io0 - 5 Years ago In My Sweet 15 ', 1]]

#### Import modules

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#### Train Test split

In [0]:
data = []
labels =  []

for row in sub_sample:
  data.append(row[0])
  labels.append(row[1])

split = int(test_split * train_size)
train_data = data[:split]
train_labels = labels[:split]

test_data = data[split:]
test_labels = labels[split:]


In [13]:
train_data[:5]

["@evanl1 Don't tweet about food. I'm so hungry ",
 "@moeneekah ok, I have to like Mish it and try and finish my assingment tonight so I can't go out tomorrow  (that means no schoolll) yew ha",
 'Just got a firmware update on his G1...looks good so far ',
 'omg! my stomach hurts so badly! i hope Lauren didnt give me what she had! ',
 'http://twitpic.com/68io0 - 5 Years ago In My Sweet 15 ']

In [14]:
print(len(train_data))
print(len(train_labels))
print(len(test_data))
print(len(test_labels))

144000
144000
16000
16000


In [15]:
set(train_labels)

{0, 1}

#### Tokenize data

In [0]:
tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index

train_seq = tokenizer.texts_to_sequences(train_data)
train_pad = pad_sequences(train_seq, maxlen=max_len, padding=pad_type, truncating=trunc_type)

test_seq = tokenizer.texts_to_sequences(test_data)
test_pad = pad_sequences(test_seq, maxlen=max_len, padding=pad_type, truncating=trunc_type)

In [17]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)

In [19]:
print(train_labels_final.shape)
print(train_pad.shape)

(144000, 2)
(144000, 32)


In [20]:

vocab_size = len(word_index)
print(vocab_size)

128439


#### Get word embeddings

In [21]:
# Note this is the 100 dimension version of GloVe from Stanford
# I unzipped and hosted it on my site to make this notebook easier
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

--2020-04-07 15:46:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M  62.6MB/s    in 5.3s    

2020-04-07 15:46:37 (62.6 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [22]:
print(len(embeddings_matrix))

128440


#### Build a model and fit it

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_len, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 100)           12844000  
_________________________________________________________________
dropout (Dropout)            (None, 32, 100)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 28, 64)            32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 7, 64)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 12,909,283
Trainable params: 65,283
Non-trainable params: 12,844,000
_______________________________________

In [27]:
num_epochs = 50
history = model.fit(train_seq, train_labels_final, epochs=num_epochs, validation_data=(test_seq, test_labels_final), verbose=2)

print("Training Complete")

ValueError: ignored